In [7]:
from typing import Tuple
import collections 

## HEAP

In [8]:
class Heap:

    def __init__(self) -> None:
        self.tree = []

    def _parent(self, ix: int) -> int:
        return (ix - 1) // 2

    def _children(self, ix: int) -> Tuple[int, int]:
        # 0 -> 1,2  (2*0+1, 2*0+2)
        # 1 -> 3,4  (2*1+1, 2*1+2)
        # 2 -> 5,6  (2*2+1, 2*2+2)
        return (2*ix+1, 2*ix+2)

    def _safe_ix_val(self, ix: int) -> float:
        return  self.tree[ix] if ix < len(self.tree) else float('inf')

    def insert(self, val: float) -> None:
        """Insert a new element and maintain invariant with Bubble Up Algo.
        
        Append the new value to the end of the list. Then keep swapping it
        with the parent until the parent of the final position is smaller
        than the element.
        """
        self.tree.append(val)
        ix = len(self.tree) - 1
        p_ix = self._parent(ix)
        # Bubble up until until heap invariant is met: parent <= child
        while self.tree[p_ix] > self.tree[ix]:
            self.tree[p_ix], self.tree[ix] = self.tree[ix], self.tree[p_ix]
            ix = p_ix
            p_ix = self._parent(ix)
            if p_ix < 0:
                break
    
    def extract_min(self) -> float:
        """Pluck the smallest and maintain invariant with Bubble Down algo.
        
        Element at index 0 is the smallest element. Pop out the last element
        of the list and assign it to index 0. Now keep swapping the smaller
        of the two children until the parent is smaller than both children.
        """
        if len(self.tree) == 1:
            return self.tree.pop()
        minval = self.tree[0]
        # assign the last value to the root
        self.tree[0] = self.tree.pop()
        ix = 0
        lt, rt = self._children(ix)
        lt_val = self._safe_ix_val(lt)
        rt_val = self._safe_ix_val(rt)
        # Bubble down the value until heap invariant is met: parent <= child
        while (self.tree[ix] > lt_val) or (self.tree[ix] > rt_val):
            min_child_ix = lt if lt_val < rt_val else rt
            self.tree[ix], self.tree[min_child_ix] = self.tree[min_child_ix], self.tree[ix]
            ix = min_child_ix
            lt, rt = self._children(min_child_ix)
            lt_val = self._safe_ix_val(lt)
            rt_val = self._safe_ix_val(rt)
        return minval

    def print(self) -> None:
        # Level order traversal
        if not self.tree:
            print(None)
            return
        q = collections.deque([0])
        ntree = len(self.tree)
        while q:
            n = len(q)
            for _ in range(n):
                ix = q.popleft()
                print(f"{self.tree[ix]} ", end="") 
                lt, rt = self._children(ix)
                if lt < ntree:
                    q.append(lt)
                if rt < ntree:
                    q.append(rt)
            print()

In [9]:
hp = Heap()
hp.insert(1)
hp.insert(3)
hp.insert(0)
hp.print()
hp.insert(10)
hp.insert(3)
hp.insert(4)
hp.insert(5)
print(hp.extract_min())
print()
hp.print()
hp.insert(6)
hp.insert(3)
hp.insert(1)
hp.insert(0)
hp.print()

0 
3 1 
0

1 
3 4 
10 3 5 
0 
1 4 
3 1 5 6 
10 3 3 


In [10]:
while True:
    try:
        print(hp.extract_min(), end=" ")
    except IndexError:
        break
print("\n----")
hp.print()

0 1 1 3 3 3 4 5 6 10 
----
None


## TRIES

In [3]:
"""208. Implement Trie (Prefix Tree)

https://leetcode.com/problems/implement-trie-prefix-tree/

Logic
-----
Use a dict of child_char -> children_dict. Add characters iteratively
and find iteratively. Start with a sentinel root node.

NOTE: Original implementation uses list/arrays and not dicts. This can
be easily converted to a list of lists.

words: apple, app, be

Root node   {'a':  , 'b':}
              /        \
          {'p':}        {'e': }
           /                \
         {'p':}             {'E': {}}
         /
        {'l': , 'E': {}}
        /
      {'e':}
      /
      {'E': {}} 
The trie is maintained by a dict where each key is
a child character which contains a dict of it's child_char -> children_dict.

Every time we get a new a word gets added
1. We add `E` to the end.
2. We iterate over children of root node and keep moving until we 
    have matching children. The moment we run out of matching children
    we add new nodes.

`Search` we add `E` to the word. We iterate over node until
    both match if they stop matching we return False.
"""

class Trie:

    def __init__(self):
        self.root = {}

    def insert(self, word: str) -> None:
        """
        R - a - p - p - l - e - E
                \
                 l - e - E
        Tests
        =====
        1. appE, wlen = 4
           0123
        node    ix  c
        {S}     0   a
        {a}     1   p
        {p}     2   p
        {p }    3   E
        {E}     4

        2. apleE, wlen = 5
           01234
        node    ix  c
        {S}     0   a
        {a}     1   p
        {p}     2   l
        {l}
        """
        word = word + "E"
        ix, node, wlen = 0, self.root, len(word)
        # Walk through existing nodes
        while ix < wlen:
            c = word[ix]
            if c not in node:
                break
            node = node[c]
            ix += 1
        # Create missing nodes
        while ix < wlen:
            c = word[ix]
            node[c] = {}
            node = node[c]
            ix += 1

    def search(self, word: str) -> bool:
        return self.startsWith(prefix=word+'E')

    def startsWith(self, prefix: str) -> bool:
        """
        Tests
        =====
        1. app: plen = 3
        012
        node    ix  c
        {S}     0   a
        {a}     1   p
        {p}     2   p
        {p}     3
        
        2. al: plen = 2
        01
        node    ix  c
        {S}     0   a
        {a}     1   l
        {l}
        """
        node = self.root
        for c in prefix:
            if c not in node:
                return False
            node = node[c]
        else:
            # We iterated through the entire word and found no
            # missing character
            return True

In [4]:
trie = Trie()
trie.insert("apple")
print(trie.search("apple"))   # return True
print(trie.search("app"))     # return False
print(trie.startsWith("app")) # return True
trie.insert("app")
print(trie.search("app"))     # return True

True
False
True
True


### DISJOINT SET UNION (DSU)

In [7]:
class DSU:

    def __init__(self, n: int) -> None:
        self.parent = list(range(n))
        self.size = [1] * n

    def find_leader(self, x: int) -> int:
        if self.parent[x] == x:
            return x
        self.parent[x] = self.find_leader(self.parent[x])
        return self.parent[x]

    def union(self, x: int, y: int) -> None:
        xlead = self.find_leader(x)
        ylead = self.find_leader(y)
        if xlead == ylead:
            return
        if self.size[xlead] >= self.size[ylead]:
            self.parent[ylead] = xlead
            self.size[xlead] += self.size[ylead]
        else:
            self.parent[xlead] = ylead
            self.size[ylead] += self.size[xlead]

    def is_connected(self, x: int, y: int) -> bool:
        return self.find_leader(x) == self.find_leader(y)

In [27]:
dsu = DSU(n=10)
_print()
for x, y in [
    (2, 3),
    (2, 3),
    (4, 3),
    (4, 9),
    (5, 7),
]:
    print(f"Union({x}, {y})")
    dsu.union(x=x,y=y)
    print(f"dsu.parent:\t{dsu.parent}")
    print(f"dsu.size:\t{dsu.size}")
    print()

print(f"{dsu.is_connected(5,7)=}")
print(f"{dsu.is_connected(4,3)=}")
print(f"{dsu.is_connected(5,3)=}")

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Union(2, 3)
dsu.parent:	[0, 1, 2, 2, 4, 5, 6, 7, 8, 9]
dsu.size:	[1, 1, 2, 1, 1, 1, 1, 1, 1, 1]

Union(2, 3)
dsu.parent:	[0, 1, 2, 2, 4, 5, 6, 7, 8, 9]
dsu.size:	[1, 1, 2, 1, 1, 1, 1, 1, 1, 1]

Union(4, 3)
dsu.parent:	[0, 1, 2, 2, 2, 5, 6, 7, 8, 9]
dsu.size:	[1, 1, 3, 1, 1, 1, 1, 1, 1, 1]

Union(4, 9)
dsu.parent:	[0, 1, 2, 2, 2, 5, 6, 7, 8, 2]
dsu.size:	[1, 1, 4, 1, 1, 1, 1, 1, 1, 1]

Union(5, 7)
dsu.parent:	[0, 1, 2, 2, 2, 5, 6, 5, 8, 2]
dsu.size:	[1, 1, 4, 1, 1, 2, 1, 1, 1, 1]

dsu.is_connected(5,7)=True
dsu.is_connected(4,3)=True
dsu.is_connected(5,3)=False
